### Mod 1 Project --- Data Movie

In [1]:
#top 50 lucrative movies from  tn_movie_budgets and bom_movie_gross   (find their ratings)
# most (top50) popular movies from tmdb_movies
# highest(top 50) ratings from imdb_title_ratings

# select common movies from above list
#find their genre, directors

### Import libraries & database

In [2]:
!pip install mysql-connector-python

In [3]:
import pandas as pd
import numpy as np
import mysql.connector
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [28]:
con = mysql.connector.connect(user='hbs-student', password='super-secure-password?',
                              host='hbs-ds-060120.c1doesqrid0e.us-east-1.rds.amazonaws.com',
                              database='movies')
cur = con.cursor()

# Data Cleaning

### IMDB Titles

In [5]:
query = ('''
SELECT *
FROM imdb_title_basics imdb_b
JOIN imdb_title_crew imdb_c USING (tconst)
JOIN imdb_title_ratings imdb_e USING (tconst)
''')
imdb_titles = pd.read_sql_query(query, con)
#joining all IMDB title tables using title ids and name ids

In [6]:
imdb_titles.isna().sum()
#checking for null values

tconst                 0
primary_title          0
original_title         0
start_year             0
runtime_minutes     7620
genres                 0
directors              0
writers            10561
averagerating          0
numvotes               0
dtype: int64

In [7]:
imdb_titles = imdb_titles.dropna(subset=['runtime_minutes'])
imdb_titles = imdb_titles.dropna(subset=['writers'])
#Removing rows with null values for runtime minutes and writers

In [8]:
pd.set_option('display.max_columns',None)
# Want to see all columns to see which are going to be useful

In [9]:
imdb_titles['genres'] = imdb_titles['genres'].map(lambda x: x.split(','))
imdb_titles['directors'] = imdb_titles['directors'].map(lambda x: x.split(','))
imdb_titles['writers'] = imdb_titles['writers'].map(lambda x: x.split(','))
# Converting genres, directors, writers to lists, so we can easily check most popular later.

In [10]:
imdb_titles.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,directors,writers,averagerating,numvotes
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"[Action, Crime, Drama]",[nm0712540],"[nm0023551, nm1194313, nm0347899, nm1391276]",7.0,77
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,[Drama],[nm0000080],"[nm0000080, nm0462648]",6.9,4517
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"[Comedy, Drama, Fantasy]","[nm0765384, nm0749914]","[nm1360635, nm0749914]",6.5,119
6,tt0137204,Joe Finds Grace,Joe Finds Grace,2017,83.0,"[Adventure, Animation, Comedy]",[nm0365480],[nm0365480],8.1,263
7,tt0146592,Pál Adrienn,Pál Adrienn,2010,136.0,[Drama],[nm1030585],"[nm1030585, nm1921430]",6.8,451


In [11]:
imdb_titles['age_of_movie']= 2020 - imdb_titles['start_year']
#Creating new column to have "age of movie" as a reference

### IMDB Crew

In [12]:
query = ('''
SELECT *
FROM imdb_name_basics
''')
imdb_crew = pd.read_sql_query(query, con)

In [13]:
imdb_crew.isna().sum()
# Checking for null values

nconst                     0
primary_name               0
birth_year            523912
death_year            599865
primary_profession         0
known_for_titles           0
dtype: int64

In [14]:
imdb_crew = imdb_crew.fillna(0)
#Replacing null values with 0

In [15]:
imdb_crew['birth_year'] = imdb_crew['birth_year'].map(lambda x: int(x))
imdb_crew['death_year'] = imdb_crew['death_year'].map(lambda x: int(x))
#Converting birth years and death years to integer

In [16]:
imdb_crew.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606648 entries, 0 to 606647
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   nconst              606648 non-null  object
 1   primary_name        606648 non-null  object
 2   birth_year          606648 non-null  int64 
 3   death_year          606648 non-null  int64 
 4   primary_profession  606648 non-null  object
 5   known_for_titles    606648 non-null  object
dtypes: int64(2), object(4)
memory usage: 27.8+ MB


In [17]:
imdb_crew['primary_profession'] = imdb_crew['primary_profession'].map(lambda x: x.split(','))
imdb_crew['known_for_titles'] = imdb_crew['known_for_titles'].map(lambda x: x.split(','))
#Converting these to lists so we can easily reference them later

In [18]:
imdb_crew.head()

,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
0,nm0000002,Lauren Bacall,1924,2014,"[actress, soundtrack]","[tt0038355, tt0117057, tt0071877, tt0037382]"
1,nm0000003,Brigitte Bardot,1934,0,"[actress, soundtrack, producer]","[tt0049189, tt0057345, tt0054452, tt0059956]"
2,nm0000005,Ingmar Bergman,1918,2007,"[writer, director, actor]","[tt0069467, tt0050986, tt0083922, tt0050976]"
3,nm0000006,Ingrid Bergman,1915,1982,"[actress, soundtrack, producer]","[tt0071877, tt0036855, tt0038109, tt0038787]"
4,nm0000007,Humphrey Bogart,1899,1957,"[actor, soundtrack, producer]","[tt0034583, tt0033870, tt0037382, tt0043265]"


In [24]:
imdb_crew['age'] = imdb_crew.apply(lambda row: (2020 - row['birth_year']) if row['death_year'] == 0 else None, axis=1)
#Adding an age column. If the age is 2020, it means that there is no birth year available. If the age is equal to None, it means the person is dead.

In [26]:
imdb_crew.head(100)

,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles,age
0,nm0000002,Lauren Bacall,1924,2014,"[actress, soundtrack]","[tt0038355, tt0117057, tt0071877, tt0037382]",NaN
1,nm0000003,Brigitte Bardot,1934,0,"[actress, soundtrack, producer]","[tt0049189, tt0057345, tt0054452, tt0059956]",86.0
2,nm0000005,Ingmar Bergman,1918,2007,"[writer, director, actor]","[tt0069467, tt0050986, tt0083922, tt0050976]",NaN
3,nm0000006,Ingrid Bergman,1915,1982,"[actress, soundtrack, producer]","[tt0071877, tt0036855, tt0038109, tt0038787]",NaN
4,nm0000007,Humphrey Bogart,1899,1957,"[actor, soundtrack, producer]","[tt0034583, tt0033870, tt0037382, tt0043265]",NaN
...,...,...,...,...,...,...,...
95,nm0000149,Jodie Foster,1962,0,"[actress, director, producer]","[tt0118884, tt0476964, tt0102926, tt0094608]",58.0
96,nm0000150,Michael J. Fox,1961,0,"[actor, producer, miscellaneous]","[tt0116365, tt0088763, tt0099088, tt0096874]",59.0
97,nm0000151,Morgan Freeman,1937,0,"[actor, producer, soundtrack]","[tt1057500, tt0097239, tt0405159, tt0114369]",83.0
98,nm0000152,Richard Gere,1949,0,"[actor, soundtrack, producer]","[tt0119395, tt0299658, tt0100405, tt0358135]",71.0


### BOM

In [29]:
query = ('''
SELECT * FROM bom_movie_gross
''')
bom = pd.read_sql_query(query, con)

In [30]:
#drop Null value in domestic_gross
bom = bom.dropna(subset =['domestic_gross'])

#convert Null value in foreign_gross to 0
bom.foreign_gross = bom.foreign_gross.fillna(0)

#convert domestic_gross and foreign_gross from float to int
bom.domestic_gross = bom.domestic_gross.map(lambda x: int(x))   
bom.foreign_gross = bom.foreign_gross.map(lambda x: int(x))  

In [31]:
#remove spaces on both end of strings
bom.title = bom.title.map(lambda x: x.strip())
bom.studio = bom.studio.map(lambda x: x.strip())

In [32]:
bom.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3359 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   title           3359 non-null   object
 1   studio          3359 non-null   object
 2   domestic_gross  3359 non-null   int64 
 3   foreign_gross   3359 non-null   int64 
 4   year            3359 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 157.5+ KB


### TN

In [34]:
query_tn = ('''
SELECT * FROM tn_movie_budgets
''')
tn = pd.read_sql_query(query_tn, con)

In [35]:
# convert release_date to date format
tn.release_date = tn.release_date.map(lambda x: dt.datetime.strptime(x, '%b %d, %Y'))

# convert production_budget, domestic_gross, worldwide_gross to date format
tn.production_budget = tn.production_budget.map(lambda x: int(x.replace(',','').replace('$','')))
tn.domestic_gross = tn.domestic_gross.map(lambda x: int(x.replace(',','').replace('$','')))
tn.worldwide_gross = tn.worldwide_gross.map(lambda x: int(x.replace(',','').replace('$','')))

#add profit column
tn['profit'] = tn.worldwide_gross - tn.production_budget

In [36]:
#remove spaces on both end of strings
tn.movie = tn.movie.map(lambda x: x.strip())

In [37]:
tn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 5782 non-null   int64         
 1   release_date       5782 non-null   datetime64[ns]
 2   movie              5782 non-null   object        
 3   production_budget  5782 non-null   int64         
 4   domestic_gross     5782 non-null   int64         
 5   worldwide_gross    5782 non-null   int64         
 6   profit             5782 non-null   int64         
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 316.3+ KB


### TMDB

In [39]:
query_tmdb = ('''
SELECT * FROM tmdb_movies
''')
tmdb = pd.read_sql_query(query_tmdb, con)

In [40]:
query_tmdb_link = ('''
SELECT * FROM tmdb_imdb_link
''')
tmdb_link = pd.read_sql_query(query_tmdb_link, con)

In [41]:
#convert genre_ids to int
tmdb.genre_ids.map(lambda x: int(x))

#remove spaces on both end of strings
tmdb.original_language = tmdb.original_language.map(lambda x: x.strip())
tmdb.original_title = tmdb.original_title.map(lambda x: x.strip())
tmdb.title = tmdb.title.map(lambda x: x.strip())

In [42]:
#Join tmdb and tmdb_link tables
tmdb = tmdb.join(tmdb_link.set_index('tmdb_id'), on = 'id', how = 'left')

In [43]:
#clean the joined table
tmdb.imdb_id = tmdb.imdb_id.fillna(0).map(lambda x: int(x))

In [44]:
tmdb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26526 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   genre_ids          26526 non-null  object 
 1   id                 26526 non-null  int64  
 2   original_language  26526 non-null  object 
 3   original_title     26526 non-null  object 
 4   popularity         26526 non-null  float64
 5   release_date       26526 non-null  object 
 6   title              26526 non-null  object 
 7   vote_average       26526 non-null  float64
 8   vote_count         26526 non-null  int64  
 9   imdb_id            26526 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.2+ MB


### RT

In [46]:
query_rt_info = ('''
SELECT * FROM rt_movie_info
''')
rt_info = pd.read_sql_query(query_rt_info, con)

In [47]:
query_rt_reviews = ('''
SELECT * FROM rt_reviews
''')
rt_reviews = pd.read_sql_query(query_rt_reviews, con)

In [48]:
query_rt_link = ('''
SELECT * FROM rt_imdb_link
''')
rt_link = pd.read_sql_query(query_rt_link, con)

In [49]:
rt_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1560 non-null   object
 2   rating        1560 non-null   object
 3   genre         1560 non-null   object
 4   director      1560 non-null   object
 5   writer        1560 non-null   object
 6   theater_date  1560 non-null   object
 7   dvd_date      1560 non-null   object
 8   currency      1560 non-null   object
 9   box_office    1560 non-null   object
 10  runtime       1560 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


### RT_Info Cleaning

In [51]:
#replace empty value in theater_date and dvd_date with Jan 1,1000
rt_info['theater_date'].loc[rt_info.theater_date == ''] = 'Jan 1, 1000'
rt_info['dvd_date'].loc[rt_info.dvd_date == ''] = 'Jan 1, 1000'

C:\Users\yazan\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [52]:
#convert theater_date and dvd_date to datetime format
rt_info.theater_date = rt_info.theater_date.map(lambda x: dt.datetime.strptime(x, '%b %d, %Y').date())
rt_info.dvd_date = rt_info.dvd_date.map(lambda x: dt.datetime.strptime(x, '%b %d, %Y').date())

In [53]:
#clean box_office date
rt_info['box_office'].loc[rt_info.box_office == ''] = '0'
rt_info['box_office'] = rt_info.box_office.map(lambda x: int(x.replace(',', '')))

In [54]:
#clean runtime (replace empty value with 0)
rt_info['runtime'].loc[rt_info.runtime == ''] = '0'
rt_info['runtime'] = rt_info.runtime.map(lambda x: int(x.replace('minutes', '')))

In [55]:
#remove spaces on both end of strings
rt_info.synopsis = rt_info.synopsis.map(lambda x: x.strip())
rt_info.rating = rt_info.rating.map(lambda x: x.strip())
rt_info.genre = rt_info.genre.map(lambda x: x.strip())
rt_info.director = rt_info.director.map(lambda x: x.strip())
rt_info.writer = rt_info.writer.map(lambda x: x.strip())

In [58]:
rt_info['genre'] = rt_info['genre'].map(lambda x: x.split('|'))
rt_info['writer'] = rt_info['writer'].map(lambda x: x.split('|'))
rt_info['director'] = rt_info['director'].map(lambda x: x.split('|'))
#Converting genre, writer, director to list so we can reference them easily

In [64]:
rt_info = rt_info.drop(columns=['box_office','currency'],axis=1)
#Dropping box_office because we have better data on another table, dropping currency because we removed box office data

### RT_Reviews Cleaning

In [65]:
#convert date to datetime format
rt_reviews.date = rt_reviews.date.map(lambda x: dt.datetime.strptime(x, '%B %d, %Y').date())

In [66]:
#convert top_critic to int
rt_reviews.top_critic = rt_reviews.top_critic.map(lambda x: int(x))

In [67]:
#remove spaces on both end of strings
rt_reviews.review = rt_reviews.review.map(lambda x: x.strip())
rt_reviews.rating = rt_reviews.rating.map(lambda x: x.strip())
rt_reviews.fresh = rt_reviews.fresh.map(lambda x: x.strip())
rt_reviews.critic = rt_reviews.critic.map(lambda x: x.strip())
rt_reviews.publisher = rt_reviews.publisher.map(lambda x: x.strip())

In [68]:
rt_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12300 entries, 0 to 12299
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          12300 non-null  int64 
 1   review      12300 non-null  object
 2   rating      12300 non-null  object
 3   fresh       12300 non-null  object
 4   critic      12300 non-null  object
 5   top_critic  12300 non-null  int64 
 6   publisher   12300 non-null  object
 7   date        12300 non-null  object
dtypes: int64(2), object(6)
memory usage: 768.9+ KB


### RT_link cleaning

In [74]:
rt_link.year = rt_link.year.fillna(1000)

In [75]:
rt_link.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17468 entries, 0 to 17467
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   imdbid       17468 non-null  int64  
 1   rtid         17468 non-null  int64  
 2   title        17468 non-null  object 
 3   year         17468 non-null  float64
 4   mpaa_rating  17468 non-null  object 
 5   runtime      17468 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 818.9+ KB


In [76]:
#replace empty space to year 1000 in year
rt_link.year = rt_link.year.fillna(1000)

#convert date to datetime format
rt_link.year = rt_link.year.map(lambda x: (int(x)))

In [77]:
#replace empty space in runtime to 0
rt_link['runtime'].loc[rt_link.runtime == ''] = '0'

#convert runtime to int format
rt_link.runtime = rt_link.runtime.map(lambda x: (int(x)))

C:\Users\yazan\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [78]:
rt_link.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17468 entries, 0 to 17467
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   imdbid       17468 non-null  int64 
 1   rtid         17468 non-null  int64 
 2   title        17468 non-null  object
 3   year         17468 non-null  int64 
 4   mpaa_rating  17468 non-null  object
 5   runtime      17468 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 818.9+ KB


# Top 200 Profitable Movies after 2000

### BOM Database

In [56]:
# get top 200 lucrative domestic movies
bom_top200_domestic = bom.sort_values('domestic_gross', ascending = False).head(200)   

### TN Database

In [57]:
# get top 200 revenue domestic movies
tn_top200_domestic = tn.sort_values('domestic_gross',ascending=False).head(200)

# get top 200 revenue worldwide movies
tn_top200_world = tn.sort_values('worldwide_gross',ascending=False).head(200)

# get top 200 profit movies
tn_top200_profit = tn.sort_values('profit',ascending=False).head(200)

# Top 200 Highest Rated Movies after 2000